#### Examining the structure of categorical inputs
For this exercise you will call ```model.matrix()``` to examine how R represents data with both categorical and numerical inputs for modeling. The dataset flowers (derived from the Sleuth3 package) is loaded into your workspace. It has the following columns:

Flowers: the average number of flowers on a meadowfoam plant
Intensity: the intensity of a light treatment applied to the plant
Time: A categorical variable - when (Late or Early) in the lifecycle the light treatment occurred
The ultimate goal is to predict Flowers as a function of Time and Intensity.

In [ ]:
# Call str on flowers to see the types of each column
str(flowers)
# 'data.frame':	24 obs. of  3 variables:
#  $ Flowers  : num  62.3 77.4 55.3 54.2 49.6 61.9 39.4 45.7 31.3 44.9 ...
#  $ Time     : chr  "Late" "Late" "Late" "Late" ...
#  $ Intensity: int  150 150 300 300 450 450 600 600 750 750 ...

# Use unique() to see how many possible values Time takes
unique(flowers$Time)
# [1] "Late"  "Early"

# Build a formula to express Flowers as a function of Intensity and Time: fmla. Print it
(fmla <- as.formula("Flowers ~ Intensity + Time"))
Flowers ~ Intensity + Time

# Use fmla and model.matrix to see how the data is represented for modeling
mmat <- model.matrix(fmla, data = flowers)

# Examine the first 20 lines of flowers
head(flowers, 20)
#    Flowers  Time Intensity
# 1     62.3  Late       150
# 2     77.4  Late       150
# 3     55.3  Late       300
# 4     54.2  Late       300
# 5     49.6  Late       450
# 6     61.9  Late       450
# 7     39.4  Late       600
# 8     45.7  Late       600
# 9     31.3  Late       750
# 10    44.9  Late       750
# 11    36.8  Late       900
# 12    41.9  Late       900
# 13    77.8 Early       150
# 14    75.6 Early       150
# 15    69.1 Early       300
# 16    78.0 Early       300
# 17    57.0 Early       450
# 18    71.1 Early       450
# 19    62.9 Early       600
# 20    52.2 Early       600

# Examine the first 20 lines of mmat
head(mmat, 20)
#    (Intercept) Intensity TimeLate
# 1            1       150        1
# 2            1       150        1
# 3            1       300        1
# 4            1       300        1
# 5            1       450        1
# 6            1       450        1
# 7            1       600        1
# 8            1       600        1
# 9            1       750        1
# 10           1       750        1
# 11           1       900        1
# 12           1       900        1
# 13           1       150        0
# 14           1       150        0
# 15           1       300        0
# 16           1       300        0
# 17           1       450        0
# 18           1       450        0
# 19           1       600        0
# 20           1       600        0

#### Modeling with categorical inputs
For this exercise you will fit a linear model to the flowers data, to predict Flowers as a function of Time and Intensity.

The model formula fmla that you created in the previous exercise is still in your workspace, as is the model matrix mmat.

In [ ]:
# Fit a model to predict Flowers from Intensity and Time : flower_model
flower_model <- lm(fmla, data = flowers)

# Use summary to examine flower_model 
summary(flower_model)

# Predict the number of flowers on each plant
flowers$predictions <- predict(flower_model, flowers)
# Call:
# lm(formula = fmla, data = flowers)

# Residuals:
#    Min     1Q Median     3Q    Max 
# -9.652 -4.139 -1.558  5.632 12.165 

# Coefficients:
#               Estimate Std. Error t value Pr(>|t|)    
# (Intercept)  83.464167   3.273772  25.495  < 2e-16 ***
# Intensity    -0.040471   0.005132  -7.886 1.04e-07 ***
# TimeLate    -12.158333   2.629557  -4.624 0.000146 ***
# ---
# Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

# Residual standard error: 6.441 on 21 degrees of freedom
# Multiple R-squared:  0.7992,	Adjusted R-squared:   0.78 
# F-statistic: 41.78 on 2 and 21 DF,  p-value: 4.786e-08

# Plot predictions vs actual flowers (predictions on x-axis)
ggplot(flowers, aes(x = predictions, y = Flowers)) + 
  geom_point() +
  geom_abline(color = "blue") 

![categorical_inputs_1](./figures/categorical_inputs_1.png)

#### Modeling an interaction
In this exercise you will use interactions to model the effect of gender and gastric activity on alcohol metabolism.

The data frame alcohol has columns:

- Metabol: the alcohol metabolism rate
- Gastric: the rate of gastric alcohol dehydrogenase activity
- Sex: the sex of the drinker (Male or Female)

In the video, we fit three models to the alcohol data:

- one with only additive (main effect) terms : 
    - ```Metabol ~ Gastric + Sex```
- two models, each with interactions between gastric activity and sex 

We saw that one of the models with interaction terms had a better R-squared than the additive model, suggesting that using interaction terms gives a better fit. In this exercise we will compare the R-squared of one of the interaction models to the main-effects-only model.

Recall that the operator : designates the interaction between two variables. The operator * designates the interaction between the two variables, plus the main effects.

```
x*y = x + y + x:y
```

In [ ]:
# alcohol is in the workspace
summary(alcohol)
#     Subject         Metabol          Gastric          Sex    
#  Min.   : 1.00   Min.   : 0.100   Min.   :0.800   Female:18  
#  1st Qu.: 8.75   1st Qu.: 0.600   1st Qu.:1.200   Male  :14  
#  Median :16.50   Median : 1.700   Median :1.600              
#  Mean   :16.50   Mean   : 2.422   Mean   :1.863              
#  3rd Qu.:24.25   3rd Qu.: 2.925   3rd Qu.:2.200              
#  Max.   :32.00   Max.   :12.300   Max.   :5.200              
#           Alcohol  
#  Alcoholic    : 8  
#  Non-alcoholic:24

# Create the formula with main effects only
(fmla_add <- Metabol ~ Gastric + Sex )
# Metabol ~ Gastric + Sex

# Create the formula with interactions
(fmla_interaction <- Metabol ~ Gastric + Gastric : Sex )
# Metabol ~ Gastric + Gastric:Sex

# Fit the main effects only model
model_add <- lm(fmla_add, alcohol)

# Fit the interaction model
model_interaction <- lm(fmla_interaction, alcohol)

# Call summary on both models and compare
summary(model_add)
# Call:
# lm(formula = fmla_add, data = alcohol)

# Residuals:
#     Min      1Q  Median      3Q     Max 
# -2.2779 -0.6328 -0.0966  0.5783  4.5703 

# Coefficients:
#             Estimate Std. Error t value Pr(>|t|)    
# (Intercept)  -1.9466     0.5198  -3.745 0.000796 ***
# Gastric       1.9656     0.2674   7.352 4.24e-08 ***
# SexMale       1.6174     0.5114   3.163 0.003649 ** 
# ---
# Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

# Residual standard error: 1.331 on 29 degrees of freedom
# Multiple R-squared:  0.7654,	Adjusted R-squared:  0.7492 
# F-statistic: 47.31 on 2 and 29 DF,  p-value: 7.41e-10

summary(model_interaction)
# Call:
# lm(formula = fmla_interaction, data = alcohol)

# Residuals:
#     Min      1Q  Median      3Q     Max 
# -2.4656 -0.5091  0.0143  0.5660  4.0668 

# Coefficients:
#                 Estimate Std. Error t value Pr(>|t|)    
# (Intercept)      -0.7504     0.5310  -1.413 0.168236    
# Gastric           1.1489     0.3450   3.331 0.002372 ** 
# Gastric:SexMale   1.0422     0.2412   4.321 0.000166 ***
# ---
# Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

# Residual standard error: 1.204 on 29 degrees of freedom
# Multiple R-squared:  0.8081,	Adjusted R-squared:  0.7948 
# F-statistic: 61.05 on 2 and 29 DF,  p-value: 4.033e-11

#### Modeling an interaction (2)
In this exercise, you will compare the performance of the interaction model you fit in the previous exercise to the performance of a main-effects only model. Because this data set is small, we will use cross-validation to simulate making predictions on out-of-sample data.

You will begin to use the dplyr package to do calculations.

- mutate() adds new columns to a tbl (a type of data frame)
- group_by() specifies how rows are grouped in a tbl
- summarize() computes summary statistics of a column

You will also use tidyr's gather() which takes multiple columns and collapses them into key-value pairs.

In [ ]:
# Create the splitting plan for 3-fold cross validation
set.seed(34245)  # set the seed for reproducibility
splitPlan <- kWayCrossValidation(nrow(alcohol), 3, NULL, NULL)

# Sample code: Get cross-val predictions for main-effects only model
alcohol$pred_add <- 0  # initialize the prediction vector
for(i in 1:3) {
  split <- splitPlan[[i]]
  model_add <- lm(fmla_add, data = alcohol[split$train, ])
  alcohol$pred_add[split$app] <- predict(model_add, newdata = alcohol[split$app, ])
}

# Get the cross-val predictions for the model with interactions
alcohol$pred_interaction <- 0 # initialize the prediction vector
for(i in 1:3) {
  split <- splitPlan[[i]]
  model_interaction <- lm(fmla_interaction, data = alcohol[split$train, ])
  alcohol$pred_interaction[split$app] <- predict(model_interaction, newdata = alcohol[split$app, ])
}

# Get RMSE
alcohol %>% 
  gather(key = modeltype, value = pred, pred_add, pred_interaction) %>%
  mutate(residuals = Metabol - pred) %>%      
  group_by(modeltype) %>%
  summarize(rmse = sqrt(mean(residuals^2)))
# # A tibble: 2 x 2
#   modeltype         rmse
#   <chr>            <dbl>
# 1 pred_add          1.64
# 2 pred_interaction  1.38

#### Relative error
In this exercise, you will compare relative error to absolute error. For the purposes of modeling, we will define relative error as

```
rel=(y−pred)/y
```
that is, the error is relative to the true outcome. You will measure the overall relative error of a model using root mean squared relative error:

```
rmse_rel=sqrt(mean(rel^2))
```

The example (toy) dataset fdata is loaded in your workspace. It includes the columns:

- y: the true output to be predicted by some model; imagine it is the amount of money a customer will spend on a visit to your store.
- pred: the predictions of a model that predicts y.
- label: categorical: whether y comes from a population that makes small purchases, or large ones.

You want to know which model does "better": the one predicting the small purchases, or the one predicting large ones.

In [ ]:
# fdata is in the workspace
summary(fdata)
#        y                 pred                      label   
#  Min.   :  -5.894   Min.   :   1.072   small purchases:50  
#  1st Qu.:   5.407   1st Qu.:   6.373   large purchases:50  
#  Median :  57.374   Median :  55.693                       
#  Mean   : 306.204   Mean   : 305.905                       
#  3rd Qu.: 550.903   3rd Qu.: 547.886                       
#  Max.   :1101.619   Max.   :1098.896

# Examine the data: generate the summaries for the groups large and small:
fdata %>% 
    group_by(label) %>%     # group by small/large purchases
    summarize(min  = min(y),   # min of y
              mean = mean(y),   # mean of y
              max  = max(y))   # max of y
# # A tibble: 2 x 4
#   label             min   mean    max
#   <fct>           <dbl>  <dbl>  <dbl>
# 1 small purchases -5.89   6.48   18.6
# 2 large purchases 96.1  606.   1102.

# Fill in the blanks to add error columns
fdata2 <- fdata %>% 
         group_by(label) %>%       # group by label
           mutate(residual = y - pred,  # Residual
                  relerr   = (y - pred)/y)  # Relative error

# Compare the rmse and rmse.rel of the large and small groups:
fdata2 %>% 
  group_by(label) %>% 
  summarize(rmse     = sqrt(mean(residual^2)),   # RMSE
            rmse.rel = sqrt(mean(relerr^2))) # Root mean squared relative error
# # A tibble: 2 x 3
#   label            rmse rmse.rel
#   <fct>           <dbl>    <dbl>
# 1 small purchases  4.01   1.25  
# 2 large purchases  5.54   0.0147

# Plot the predictions for both groups of purchases
ggplot(fdata2, aes(x = pred, y = y, color = label)) + 
  geom_point() + 
  geom_abline() + 
  facet_wrap(~ label, ncol = 1, scales = "free") + 
  ggtitle("Outcome vs prediction")

![relative_error_1](./figures/relative_error_1.png)

#### Modeling log-transformed monetary output
In this exercise, you will practice modeling on log-transformed monetary output, and then transforming the "log-money" predictions back into monetary units. The data loaded into your workspace records subjects' incomes in 2005 (Income2005), as well as the results of several aptitude tests taken by the subjects in 1981:

Arith
Word
Parag
Math
AFQT (Percentile on the Armed Forces Qualifying Test)
The data have already been split into training and test sets (income_train and income_test respectively) and are in the workspace. You will build a model of log(income) from the inputs, and then convert log(income) back into income.

In [ ]:
# Examine Income2005 in the training set
summary(income_train$Income2005)
#    Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
#      63   23000   39000   49894   61500  703637

# Write the formula for log income as a function of the tests and print it
(fmla.log <- log(Income2005) ~ Arith + Word + Parag + Math + AFQT)
# log(Income2005) ~ Arith + Word + Parag + Math + AFQT

# Fit the linear model
model.log <-  lm(fmla.log, data = income_train)

# Make predictions on income_test
income_test$logpred <- predict(model.log, income_test)
summary(income_test$logpred)
#    Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
#   9.766  10.133  10.423  10.419  10.705  11.006

# Convert the predictions to monetary units
income_test$pred.income <- exp(income_test$logpred)
summary(income_test$pred.income)
#    Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
#   17432   25167   33615   35363   44566   60217

#  Plot predicted income (x axis) vs income
ggplot(income_test, aes(x = pred.income, y = Income2005)) + 
  geom_point() + 
  geom_abline(color = "blue")

![modeling_log_transformed_monetary](./figures/modeling_log_transformed_monetary.png)

#### Comparing RMSE and root-mean-squared Relative Error
In this exercise, you will show that log-transforming a monetary output before modeling improves mean relative error (but increases RMSE) compared to modeling the monetary output directly. You will compare the results of model.log from the previous exercise to a model (model.abs) that directly fits income.

The income_train and income_test datasets are loaded in your workspace, along with your model, model.log.

Also in the workspace:

model.abs: a model that directly fits income to the inputs using the formula

```
Income2005 ~ Arith + Word + Parag + Math + AFQT
```

In [ ]:
# model.abs is in the workspace
summary(model.abs)
# Call:
# lm(formula = fmla.abs, data = income_train)

# Residuals:
#    Min     1Q Median     3Q    Max 
# -78728 -24137  -6979  11964 648573 

# Coefficients:
#             Estimate Std. Error t value Pr(>|t|)    
# (Intercept)  17516.7     6420.1   2.728  0.00642 ** 
# Arith         1552.3      303.4   5.116 3.41e-07 ***
# Word          -132.3      265.0  -0.499  0.61754    
# Parag        -1155.1      618.3  -1.868  0.06189 .  
# Math           725.5      372.0   1.950  0.05127 .  
# AFQT           177.8      144.1   1.234  0.21734    
# ---
# Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

# Residual standard error: 45500 on 2063 degrees of freedom
# Multiple R-squared:  0.1165,	Adjusted R-squared:  0.1144 
# F-statistic:  54.4 on 5 and 2063 DF,  p-value: < 2.2e-16

# Add predictions to the test set
income_test <- income_test %>%
  mutate(pred.absmodel = predict(model.abs, income_test),# predictions from model.abs
         pred.logmodel = exp(predict(model.log, income_test)))# predictions from model.log

# Gather the predictions and calculate residuals and relative error
income_long <- income_test %>% 
  gather(key = modeltype, value = pred, pred.absmodel, pred.logmodel) %>%
  mutate(residual = pred - Income2005,   # residuals
         relerr   = residual / Income2005)   # relative error

# Calculate RMSE and relative RMSE and compare
income_long %>% 
  group_by(modeltype) %>%      # group by modeltype
  summarize(rmse     = sqrt(mean(residual^2)),    # RMSE
            rmse.rel = sqrt(mean(relerr^2)))    # Root mean squared relative error
# # A tibble: 2 x 3
#   modeltype       rmse rmse.rel
#   <chr>          <dbl>    <dbl>
# 1 pred.absmodel 37448.     3.18
# 2 pred.logmodel 39235.     2.22

#### Input transforms: the "hockey stick"
In this exercise, we will build a model to predict price from a measure of the house's size (surface area). The data set houseprice has the columns:

- price : house price in units of $1000
- size: surface area

A scatterplot of the data shows that the data is quite non-linear: a sort of "hockey-stick" where price is fairly flat for smaller houses, but rises steeply as the house gets larger. Quadratics and tritics are often good functional forms to express hockey-stick like relationships. Note that there may not be a "physical" reason that price is related to the square of the size; a quadratic is simply a closed form approximation of the observed relationship.

You will fit a model to predict price as a function of the squared size, and look at its fit on the training data.

Because ^ is also a symbol to express interactions, use the function I() to treat the expression x^2 “as is”: that is, as the square of x rather than the interaction of x with itself.

```
exampleFormula = y ~ I(x^2)
```

In [ ]:
# houseprice is in the workspace
summary(houseprice)
#       size           price      
#  Min.   : 44.0   Min.   : 42.0  
#  1st Qu.: 73.5   1st Qu.:164.5  
#  Median : 91.0   Median :203.5  
#  Mean   : 94.3   Mean   :249.2  
#  3rd Qu.:118.5   3rd Qu.:287.8  
#  Max.   :150.0   Max.   :573.0

# Create the formula for price as a function of squared size
(fmla_sqr <- price ~ I(size^2))
# price ~ I(size^2)

# Fit a model of price as a function of squared size (use fmla_sqr)
model_sqr <- lm(fmla_sqr, houseprice)

# Fit a model of price as a linear function of size
model_lin <- lm(price ~ size, houseprice)

# Make predictions and compare
houseprice %>% 
    mutate(pred_lin = predict(model_lin),       # predictions from linear model
           pred_sqr = predict(model_sqr)) %>%   # predictions from quadratic model 
    gather(key = modeltype, value = pred, pred_lin, pred_sqr) %>% # gather the predictions
    ggplot(aes(x = size)) + 
       geom_point(aes(y = price)) +                   # actual prices
       geom_line(aes(y = pred, color = modeltype)) + # the predictions
       scale_color_brewer(palette = "Dark2")

![Input_transforms__the__hockey_stick](./figures/Input_transforms__the__hockey_stick.png)

#### Input transforms: the "hockey stick" (2)
In the last exercise you saw that a quadratic model seems to fit the houseprice data better than a linear model. In this exercise you will confirm whether the quadratic model would perform better on out-of-sample data. Since this data set is small, you will use cross-validation. The quadratic formula fmla_sqr that you created in the last exercise is in your workspace.

For comparison, the sample code will calculate cross-validation predictions from a linear model price ~ size.

In [ ]:
# houseprice is in the workspace
summary(houseprice)

# fmla_sqr is in the workspace
fmla_sqr

# Create a splitting plan for 3-fold cross validation
set.seed(34245)  # set the seed for reproducibility
splitPlan <- kWayCrossValidation(nrow(houseprice), 3, NULL, NULL)

# Sample code: get cross-val predictions for price ~ size
houseprice$pred_lin <- 0  # initialize the prediction vector
for(i in 1:3) {
  split <- splitPlan[[i]]
  model_lin <- lm(price ~ size, data = houseprice[split$train,])
  houseprice$pred_lin[split$app] <- predict(model_lin, newdata = houseprice[split$app,])
}

# Get cross-val predictions for price as a function of size^2 (use fmla_sqr)
houseprice$pred_sqr <- 0 # initialize the prediction vector
for(i in 1:3) {
  split <- splitPlan[[i]]
  model_sqr <- lm(fmla_sqr, data = houseprice[split$train, ])
  houseprice$pred_sqr[split$app] <- predict(model_sqr, newdata = houseprice[split$app, ])
}

# Gather the predictions and calculate the residuals
houseprice_long <- houseprice %>%
  gather(key = modeltype, value = pred, pred_lin, pred_sqr) %>%
  mutate(residuals = pred - price)

# Compare the cross-validated RMSE for the two models
houseprice_long %>% 
  group_by(modeltype) %>% # group by modeltype
  summarize(rmse = sqrt(mean(residuals^2)))
# # A tibble: 2 x 2
#   modeltype  rmse
#   <chr>     <dbl>
# 1 pred_lin   74.3
# 2 pred_sqr   63.7